<a href="https://colab.research.google.com/github/kelseymatsik/project_chd/blob/main/test_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Just copying the data cleaning done on the training set to the test set. Using cleaning from tori_data_cleaning.ipynb

In [1]:
! git clone https://github.com/kelseymatsik/project_chd

Cloning into 'project_chd'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 76 (delta 22), reused 18 (delta 18), pack-reused 51
Receiving objects: 100% (76/76), 984.20 KiB | 9.84 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("./project_chd/fhs_train.csv")
df.head()

,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1267,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,1209,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
2,2050,0,52,1.0,0,0.0,0.0,0,0,0,275.0,112.0,71.0,25.68,80,NaN,0
3,1183,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,3225,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0


In [9]:
# Renaming columns to be more intuitive
df = df.rename(columns = {'prevalentStroke': 'stroke',
                          'prevalentHyp': 'hypertensive',
                          'TenYearCHD': 'tenYearRisk',
                          'cigsPerDay': 'currentSmoker', # Note: Looking at the value counts and data dictionary for these values, we realized that they should be switched
                          'currentSmoker': 'cigsPerDay'}
                          )

# Dropping 'Unnamed: 0' column
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# Setting up temporary variables
sex = df['sex']
age = df['age']
education = df['education']
currentSmoker = df['currentSmoker']
cigsPerDay = df['cigsPerDay']
BPMeds = df['BPMeds']
stroke = df['stroke']
hypertensive = df['hypertensive']
diabetes = df['diabetes']
totChol = df['totChol']
sysBP = df['sysBP']
diaBP = df['diaBP']
BMI = df['BMI']
heartRate = df['heartRate']
glucose = df['glucose']

In [10]:
df.dropna(subset=['BPMeds'], inplace=True) # Drop NA's in BPMeds because values are binary

In [12]:
# Impute missing values with KKN Imputer, k=6
X = df.iloc[:, :-1]
y = df['tenYearRisk']

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=6)
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

df = pd.concat([X, y], axis=1)

In [13]:
# Windsorize variables
from scipy.stats.mstats import winsorize
# The limits argument specifies the proportion of data to winsorize at each end
# Winsorization replaces the bottom and top 2% of data w/ the 2nd and 98th percentile values, reducing extreme outliers
cols = ['totChol', 'diaBP', 'heartRate']
for c in cols:
    df[c] = winsorize(df[c], limits=[0.02, 0.02])

In [14]:
col = ['BMI']
for c in col:
    df[c] = winsorize(df[c], limits=[0.03, 0.03])

In [15]:
col = ['sysBP']
for c in col:
    df[c] = winsorize(df[c], limits=[0.04, 0.04])

In [16]:
col = ['glucose']
for c in col:
    df[c] = winsorize(df[c], limits=[0.06, 0.06])

In [19]:
# Mount cleaned data to Drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
df.to_csv('/content/drive/My Drive/Classes/DS 3001/DS3001 Projects/CHD Project/cleaned_chd_test_set.csv')

Mounted at /content/drive
